### Import Libraries

In [1]:
import pandas as pd
import pyodbc
import numpy as np
import ast

### Load cleaned CSV files

In [2]:
anime_df = pd.read_csv('../data/cleaned_anime.csv')
ratings_df = pd.read_csv('../data/cleaned_ratings.csv')

In [3]:
anime_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12064 entries, 0 to 12063
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   anime_id    12064 non-null  int64  
 1   name        12064 non-null  object 
 2   genre       12064 non-null  object 
 3   type        12064 non-null  object 
 4   episodes    12064 non-null  object 
 5   rating      12064 non-null  float64
 6   members     12064 non-null  int64  
 7   genre_list  12064 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 754.1+ KB


In [4]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813736 entries, 0 to 7813735
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


In [44]:
anime_values_df = anime_df.copy()

In [45]:
anime_values_df.drop(['genre','genre_list'],axis=1,inplace=True)
anime_values_df.reset_index(drop=True,inplace=True)
anime_values_df.head()

,anime_id,name,type,episodes,rating,members
0,32281,Kimi no Na wa.,Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,TV,64,9.26,793665
2,28977,Gintama°,TV,51,9.25,114262
3,9253,Steins;Gate,TV,24,9.17,673572
4,9969,Gintama&#039;,TV,51,9.16,151266


In [46]:
# Strip spaces and convert invalid entries to NaN
anime_values_df['rating'] = anime_values_df['rating'].astype(str).str.strip().str.lower()
anime_values_df['rating'] = anime_values_df['rating'].replace(
    ['', ' ', 'nan', 'NaN', 'None', 'NONE', 'null', 'NULL', '?'], 
    np.nan
)

# cast to native Python float or None
anime_values_df['rating'] = [
    None if pd.isna(v) else float(v)
    for v in anime_values_df['rating']
]


In [47]:
anime_values_df.dropna(inplace=True,ignore_index=True)
anime_values_df.isna().sum()


anime_id    0
name        0
type        0
episodes    0
rating      0
members     0
dtype: int64

### Connect to sql server

In [5]:
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-RES15OPT\SQLEXPRESS;'
    'DATABASE=AnimeDB;'
    'Trusted_Connection=yes;'
)

In [6]:
cursor = conn.cursor()

In [7]:
cursor.execute("select DB_NAME()")
print(cursor.fetchone())

('AnimeDB',)


### Insert into Anime Table

In [85]:
cursor.fast_executemany = True
rows = list(
    anime_values_df[['anime_id','name','type','episodes','rating','members']]
    .itertuples(index=False, name=None)
)
print("inserting into anime table")

cursor.executemany(
    "INSERT INTO Anime (anime_id, name, type, episodes, rating, members) "
    "VALUES (?, ?, ?, ?, ?, ?)",
    rows
)
conn.commit()
print("Anime table ETL complete!")

inserting into anime table
Anime table ETL complete!


### Insert into genres table

In [89]:
# flattens the list of lists into a single list
all_genres = anime_df.explode('genre_list').reset_index(drop=True)
genres_df = pd.DataFrame({'genre_name':all_genres['genre_list']})
genres_df.head()

,genre_name
0,drama
1,romance
2,school
3,supernatural
4,action


In [90]:
genres_df['genre_name'] = genres_df['genre_name'].str.strip()
genres_df.drop_duplicates(ignore_index=True,inplace=True)

In [91]:
genres_df= genres_df.sort_values(by='genre_name',axis=0)

In [92]:
genres_df.reset_index(drop=True,inplace=True)
genres_df.head()

,genre_name
0,action
1,adventure
2,cars
3,comedy
4,dementia


Genre ID mapping

In [93]:
print("inserting genres...")

genre_values = list(genres_df.itertuples(index=False,name=None))

cursor.executemany(
    '''
        INSERT INTO Genres(genre_name)
        VALUES (?)
    ''',genre_values
)

conn.commit()

inserting genres...


Build anime_genres mapping table

In [54]:
sql_genres = pd.read_sql("SELECT genre_id, genre_name FROM Genres", conn)

genre_map = dict(zip(sql_genres['genre_name'], sql_genres['genre_id']))
genre_map

C:\Users\NimaLama\AppData\Local\Temp\ipykernel_10024\1325128437.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_genres = pd.read_sql("SELECT genre_id, genre_name FROM Genres", conn)


{'action': 1,
 'adventure': 2,
 'cars': 3,
 'comedy': 4,
 'dementia': 5,
 'demons': 6,
 'drama': 7,
 'ecchi': 8,
 'fantasy': 9,
 'game': 10,
 'harem': 11,
 'hentai': 12,
 'historical': 13,
 'horror': 14,
 'josei': 15,
 'kids': 16,
 'magic': 17,
 'martial arts': 18,
 'mecha': 19,
 'military': 20,
 'music': 21,
 'mystery': 22,
 'parody': 23,
 'police': 24,
 'psychological': 25,
 'romance': 26,
 'samurai': 27,
 'school': 28,
 'sci-fi': 29,
 'seinen': 30,
 'shoujo': 31,
 'shoujo ai': 32,
 'shounen': 33,
 'shounen ai': 34,
 'slice of life': 35,
 'space': 36,
 'sports': 37,
 'super power': 38,
 'supernatural': 39,
 'thriller': 40,
 'unknown': 41,
 'vampire': 42,
 'yaoi': 43,
 'yuri': 44}

In [53]:
missing = set()
anime_df['genre_list'] = anime_df['genre_list'].apply(ast.literal_eval)
for g_list in anime_df['genre_list']:
    for g in g_list:
        g = g.strip()
        if g not in genre_map:
            missing.add(g)

print(missing)



set()


In [55]:

rows = []

for _, row in anime_df.iterrows():
    anime_id = row['anime_id']
    for genre in row['genre_list']:
        genre = genre.strip()
        genre_id = genre_map.get(genre)
        if genre_id:
            rows.append((anime_id, genre_id))
        else:
            print(f"missing genre:{genre}")
anime_genres_df = pd.DataFrame(rows, columns=['anime_id', 'genre_id'])
anime_genres_df.shape


(35642, 2)

In [56]:
anime_genres_df.drop_duplicates(subset=['anime_id','genre_id'],inplace=True)

In [57]:
anime_genres_df.duplicated().sum()

np.int64(0)

In [58]:
invalid_ids = anime_genres_df[~anime_genres_df['genre_id'].isin(sql_genres['genre_id'])]
invalid_ids.head()


,anime_id,genre_id


In [59]:
missing_anime_ids = anime_genres_df[~anime_genres_df['anime_id'].isin(anime_df['anime_id'])]
missing_anime_ids

,anime_id,genre_id


In [60]:
anime_genres_df.dtypes

anime_id    int64
genre_id    int64
dtype: object

In [61]:
existing = pd.read_sql("""
    SELECT anime_id, genre_id 
    FROM Anime_Genres
""", conn)

dupes = anime_genres_df.merge(existing, on=["anime_id", "genre_id"], how="inner")
dupes.head(), len(dupes)


C:\Users\NimaLama\AppData\Local\Temp\ipykernel_10024\1852355697.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  existing = pd.read_sql("""


(Empty DataFrame
 Columns: [anime_id, genre_id]
 Index: [],
 0)

Insert into anime_genres table

In [62]:
print("Inserting Anime_Genres mapping...")

cursor.fast_executemany = True
values = list(anime_genres_df.itertuples(index=False, name=None))

cursor.executemany("""
    INSERT INTO Anime_Genres (anime_id, genre_id)
    VALUES (?, ?)
""", values)

conn.commit()

print("Anime_Genres table loaded.")


Inserting Anime_Genres mapping...


Anime_Genres table loaded.


Insert into user_ratings (batch insert)

In [8]:
# Fetch valid anime IDs from SQL
cursor.execute("SELECT anime_id FROM Anime")
valid_ids = {row[0] for row in cursor.fetchall()}

ratings_df = ratings_df[ratings_df['anime_id'].isin(valid_ids)]

# Find invalid ones in ratings_df
invalid_ratings = ratings_df[~ratings_df['anime_id'].isin(valid_ids)]

print(f'number of invalid ratings:{len(invalid_ratings)}')

number of invalid ratings:0


In [10]:
ratings_df.drop_duplicates(subset=['anime_id','user_id'],inplace=True,ignore_index=True)

In [12]:
print("Inserting User_Ratings in batches...")

cursor.fast_executemany = True
batch_size = 10000

for start in range(0, len(ratings_df), batch_size):
    batch = ratings_df.iloc[start:start+batch_size]
    values = list(batch.itertuples(index=False, name=None))

    cursor.executemany("""
        INSERT INTO User_Ratings (user_id, anime_id, rating)
        VALUES (?, ?, ?)
    """, values)

conn.commit()

print("User_Ratings table loaded.")


Inserting User_Ratings in batches...
User_Ratings table loaded.


Close connection

In [13]:
cursor.close()
conn.close()

In [14]:
ratings_df.shape

(7813714, 3)